# Subtract out Quasars from KCWI datacubes

## Notebook iterates through each frame with a quasar

## Subtracts out QSO1 and QSO2 one at a time

## Specify which wavelengths in datacube are not included in psf model creation

## Specify which wavelengths in datacube do not receive qso subtraction

## All output fits files are saved to "psfsub" directory, denoted with: "psfsub"

In [2]:
# Necessary imports

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.optimize import curve_fit
import scipy.optimize as opt
from tqdm import tqdm
from scipy import signal
from scipy.constants import c
from scipy.ndimage import gaussian_filter
from scipy import ndimage, misc
from astropy.modeling import models, fitting
from astropy.wcs import WCS
import time


In [3]:
def vel_from_z(z):
    return z*c
def get_wav_obs(v,wave_rest):
    v = v*1e3
    return (v/c+1)*wave_rest
def get_redshift(obs, rest):
    
    return (obs/rest) - 1

In [4]:
def make_empirical_psf(dat,start_wav,end_wav,xc,yc,bbx,bby,mask,NB_WIDTH):
    
    MIN_ID = 0
    MAX_ID = len(dat)-1
    if(start_wav < MIN_ID):
        start_wav = MIN_ID
        psf_range = np.arange(0,NB_WIDTH,1)
    elif(end_wav > MAX_ID):
        psf_range = np.arange(MAX_ID-NB_WIDTH,MAX_ID,1)
        
    else:
        psf_range = np.arange(int(start_wav),int(end_wav),1)
    image = np.copy(dat)
    Y,X = np.shape(image[0])
    image[mask] = np.ones((len(mask),Y,X))*np.nan    
    box = image[psf_range,yc-bby:yc+bby+1,xc-bbx:xc+bbx+1]
    psf_models = np.nanmedian(box,0)
    x, y = np.shape(psf_models)
    # Divide Model by central QSO Flux
    flux = psf_models[round(x/2)-3:round(x/2)+2,round(y/2)]
    flux = np.nansum(flux)
    psf_models = psf_models/flux   
    return psf_models

def make_empirical_var_psf(dat,start_wav,end_wav,xc,yc,bbx,bby,mask,NB_WIDTH):
    MIN_ID = 0
    MAX_ID = len(dat)-1
    if(start_wav < MIN_ID):
        start_wav = MIN_ID
        psf_range = np.arange(0,NB_WIDTH,1)
    elif(end_wav > MAX_ID):
        psf_range = np.arange(MAX_ID-NB_WIDTH,MAX_ID,1)
    else:
        psf_range = np.arange(int(start_wav),int(end_wav),1)
    psf_models = []
    for i in psf_range:
        image = np.copy(dat[i,:,:])
        box = image[yc-bby:yc+bby+1,xc-bbx:xc+bbx+1]
        x, y = np.shape(box)
        if(i not in mask):
            psf_models.append(box)

    psf_models = np.array(psf_models)
    psf_models = np.nansum(psf_models,0)/(len(psf_range)**2)
    
    return psf_models





In [8]:



def QSO_PSF_SUB(number,NB_WIDTH,z,dv,disable,QSOX, QSOY, rx, ry,directory,addon):

    # number: file number
    # NB_WIDTH: Amount of wavelength layers used to generate psf model
    # z: qso redshift
    # dv: velocity window to exclude from psf model
    # disable: wavelength values where QSO subtraction is disabled
    # QSOX: x position of qso (image unit)
    # QSOY: same as x
    # rx/ry: x and y radius of psf model
    # directory: location of fits files to subtract from
    # addon: fits file addon (Ex: 'tr', 'psfsub')
    
    # directory specifies whether we grab fits files from 'trimcube/'
    # or fits files from 'psfsub/'
    
    # Ex: If we want to subtract out QSO2 after QSO1, we grab the fits file from 'psfsub/'
    # since those files have QSO1 subtracted out, but not QSO2.
    
    hdul = fits.open(directory+'/kb220131_'+number+'_'+addon+'_icubes.fits')
    DATA = hdul[0].data # data string
    FLUX = hdul[0].data
    x, y = np.shape(DATA[0])
    dat_header = hdul[0].header
    w, ra, dec = np.shape(hdul[0].data)

    first_lamb = dat_header['CRVAL3']
    spacing = dat_header['CD3_3']
    Lambda = np.arange(first_lamb, first_lamb + spacing*w, spacing)
    # hdul.close()

    # Get accurate QSO location from 2D Gaussian Fit
    
    whitelight = np.mean(DATA,0)
    whitelight /= np.sum(whitelight)

    y, x = np.shape(whitelight)
    x,y = np.meshgrid(np.arange(0, x, 1), np.arange(0, y, 1))

    
    
    # initial guess
    psf_guess = models.Gaussian2D(amplitude=1,x_mean=QSOX,y_mean=QSOY,x_stddev=1,y_stddev=3,theta=0)
    fitter = fitting.LevMarLSQFitter()
    fit = fitter(psf_guess, x, y, whitelight)
    
    QSOX = round(fit.x_mean.value)
    QSOY = round(fit.y_mean.value)
    
    hdul = fits.open(directory+'/kb220131_'+number+'_'+addon+'_vcubes.fits')
    VAR = hdul[0].data # data string
    var_header = hdul[0].header
    hdul.close()
    
    # Restrick +/- chosen velocity window from being used to generate psf model
    dv /= 2
    blue = get_wav_obs(-dv,1215.67*(1+z))
    red = get_wav_obs(dv,1215.67*(1+z))
    
    idx_low_qso = list(abs(Lambda-blue)).index(np.min(abs(Lambda-blue)))
    idx_high_qso = list(abs(Lambda-red)).index(np.min(abs(Lambda-red)))
    mask = np.arange(idx_low_qso,idx_high_qso+1,1)
    
    
    # Create PSF model for this velocity window.
    # All layers within this window will use this model
     #######
    NB_WIDTH /= 2
    NB_WIDTH = round(NB_WIDTH)
    
    # Build PSF model for velocity window near QSO redshift. This model is used at each layer within this window
    # For layers outside of this window, the normal PSF model creation is used
    if(z == 2.75):
        
        PSF1 = make_empirical_psf(DATA,550,700,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2 = make_empirical_psf(DATA,1330,1480,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_MODEL_qso = (PSF1 + PSF2) / 2

        PSF1_VAR = make_empirical_var_psf(VAR,550,700,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2_VAR = make_empirical_var_psf(VAR,1330,1480,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_VAR_MODEL_qso = (PSF1_VAR + PSF2_VAR) / 4
        
        PSF1 = make_empirical_psf(DATA,550,700,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2 = make_empirical_psf(DATA,1330,1480,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_MODEL_smg = (PSF1 + PSF2) / 2

        PSF1_VAR = make_empirical_var_psf(VAR,550,700,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2_VAR = make_empirical_var_psf(VAR,1330,1480,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_VAR_MODEL_smg = (PSF1_VAR + PSF2_VAR) / 4
        
    else:
        
        PSF1 = make_empirical_psf(DATA,idx_low_qso-NB_WIDTH,idx_low_qso,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2 = make_empirical_psf(DATA,idx_high_qso,idx_high_qso+NB_WIDTH,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_MODEL_qso = (PSF1 + PSF2) / 2

        PSF1_VAR = make_empirical_var_psf(VAR,idx_low_qso-NB_WIDTH,idx_low_qso,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2_VAR = make_empirical_var_psf(VAR,idx_high_qso,idx_high_qso+NB_WIDTH,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_VAR_MODEL_qso = (PSF1_VAR + PSF2_VAR) / 4

        z_smg = 2.674
        blue = get_wav_obs(-dv,1215.67*(1+z_smg))
        red = get_wav_obs(dv,1215.67*(1+z_smg))

        idx_low_smg = list(abs(Lambda-blue)).index(np.min(abs(Lambda-blue)))
        idx_high_smg = list(abs(Lambda-red)).index(np.min(abs(Lambda-red)))

        # make_empirical_psf(dat,start_wav,end_wav,xc,yc,bbx,bby,mask,NB_WIDTH)
        PSF1 = make_empirical_psf(DATA,idx_low_smg-NB_WIDTH,idx_low_smg,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2 = make_empirical_psf(DATA,idx_high_smg,idx_high_smg+NB_WIDTH,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_MODEL_smg = (PSF1 + PSF2) / 2

        PSF1_VAR = make_empirical_var_psf(VAR,idx_low_smg-NB_WIDTH,idx_low_smg,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF2_VAR = make_empirical_var_psf(VAR,idx_high_smg,idx_high_smg+NB_WIDTH,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
        PSF_VAR_MODEL_smg = (PSF1_VAR + PSF2_VAR) / 4
    
    #######
    
    # Restrick layers within chosen velocity window centered on SMG from being used to generate psf model
    z_smg = 2.674
    blue = get_wav_obs(-dv,1215.67*(1+z_smg))
    red = get_wav_obs(dv,1215.67*(1+z_smg))
    
    idx_low_smg = list(abs(Lambda-blue)).index(np.min(abs(Lambda-blue)))
    idx_high_smg = list(abs(Lambda-red)).index(np.min(abs(Lambda-red)))
    
    idx_range = np.arange(idx_low_smg,idx_high_smg+1,1)
    
    mask = np.append(mask,idx_range)
    mask = mask[mask >= 0]
    mask = mask.astype(int)
    
    output = np.copy(DATA)
    output_VAR = np.copy(VAR)

    # Restrick layers within chosen velocity window centered on SMG from being used to generate psf model
    z_smg = 2.674
    blue = get_wav_obs(-dv,1215.67*(1+z_smg))
    red = get_wav_obs(dv,1215.67*(1+z_smg))
    
    idx_low_smg = list(abs(Lambda-blue)).index(np.min(abs(Lambda-blue)))
    idx_high_smg = list(abs(Lambda-red)).index(np.min(abs(Lambda-red)))
    
    idx_range = np.arange(idx_low_smg,idx_high_smg+1,1)
    
    mask = np.append(mask,idx_range)
    mask = mask[mask >= 0]
    mask = mask.astype(int)
    
    output = np.copy(DATA)
    output_VAR = np.copy(VAR)
    
    

    for i in range(len(output)):
        # print(i)
#         if(i not in disable):
        if(i not in disable):

            if(i >= idx_low_qso and i <= idx_high_qso):
                PSF = PSF_MODEL_qso
                PSF_VAR = PSF_VAR_MODEL_qso
                
            elif(i >= idx_low_smg and i <= idx_high_smg):
                PSF = PSF_MODEL_smg
                PSF_VAR = PSF_VAR_MODEL_smg
            
            else:

                ls = i - NB_WIDTH # Layers used to construct psf model at this current wavelength index
                rs = i + NB_WIDTH
                PSF = make_empirical_psf(DATA,ls,rs,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
                PSF_VAR = make_empirical_var_psf(DATA,ls,rs,QSOX,QSOY,rx,ry,mask,NB_WIDTH)
            CUBE_LAYER = DATA[i][QSOY-ry:QSOY+ry+1,QSOX-rx:QSOX+rx+1]
            
            
            x, y = np.shape(PSF)
            flux = np.sum(CUBE_LAYER[round(x/2)-2:round(x/2)+3,round(y/2)])
            MODEL = PSF*flux
            CUBE_LAYER = CUBE_LAYER - MODEL
            output[i][QSOY-ry:QSOY+ry+1,QSOX-rx:QSOX+rx+1] = CUBE_LAYER

            
            CUBE_LAYER_VAR = VAR[i][QSOY-ry:QSOY+ry+1,QSOX-rx:QSOX+rx+1]
            flux = np.sum(CUBE_LAYER_VAR[int(x/2),int(y/2)])
            flux=1
            MODEL = PSF_VAR*flux
            CUBE_LAYER_VAR = CUBE_LAYER_VAR + MODEL
            output_VAR[i][QSOY-ry:QSOY+ry+1,QSOX-rx:QSOX+rx+1] = CUBE_LAYER_VAR
            
            

    hdu = fits.PrimaryHDU(data=output)
    hdu.header = dat_header
    hdu.writeto('psfsub/kb220131_'+number+'_tr_psfsub_icubes.fits',overwrite=True)
    print('Saved fits file to: psfsub/kb220131_'+number+'_tr_psfsub_icubes.fits')
    
    hdu = fits.PrimaryHDU(data=output_VAR)
    hdu.header = var_header
    hdu.writeto('psfsub/kb220131_'+number+'_tr_psfsub_vcubes.fits',overwrite=True)
    print('Saved fits file to: psfsub/kb220131_'+number+'_tr_psfsub_vcubes.fits')
    
    

In [9]:
# South Pointing
numbers = [53,54,55,56,57,58,65] # frame numbers corresponding to South 
# These "disable" arrays correspond to the layers where the QSO is absorbed, i.e. the SMG redshift and QSO2 redshift for the spectrum of QSO1
# subtraction is disabled and the index is not used to build PSF model
# User can adjust these arrays as they see fit 
# numbers = [54]
disable = []

for n in numbers:
    number = str(n)
    
    disable = np.arange(802,865,1)
    disable = np.append(disable,np.arange(980,1050,1))

    QSO_PSF_SUB(number,50,2.916,1000,disable,15,31,2,6,'trimcube','tr')
    
    disable = np.arange(812,817,1)
    disable = np.append(disable,np.arange(833,837,1))
    disable = np.append(disable,np.arange(855,859,1))

    QSO_PSF_SUB(number,50,2.75,1000,disable,23,16,2,6,'psfsub','tr_psfsub')
    

Saved fits file to: psfsub/kb220131_53_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_53_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_53_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_53_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_54_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_54_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_54_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_54_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_55_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_55_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_55_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_55_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_56_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_56_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_56_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_56_tr_psfsub_vcubes.fits
Saved fits file to: psfs

In [6]:
# North Pointing (No QSO!)
numbers = [59,61,62,63,64,66] # frame numbers corresponding to South 
# These "disable" arrays correspond to the layers where the QSO is absorbed, i.e. the SMG redshift and QSO2 redshift for the spectrum of QSO1
# subtraction is disabled and the index is not used to build PSF model
# User can adjust these arrays as they see fit 
# numbers = [54]
disable = []

for n in numbers:
    number = str(n)
    
    disable = np.arange(0,1774,1)
    
    QSO_PSF_SUB(number,0,2,1000,disable,10,10,0,0,'trimcube','tr')
    


/Users/khall12/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
<ipython-input-4-ece04bcef24c>:44: RuntimeWarning: invalid value encountered in double_scalars
  psf_models = np.nansum(psf_models,0)/(len(psf_range)**2)


Saved fits file to: psfsub/kb220131_59_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_59_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_61_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_61_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_62_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_62_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_63_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_63_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_64_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_64_tr_psfsub_vcubes.fits
Saved fits file to: psfsub/kb220131_66_tr_psfsub_icubes.fits
Saved fits file to: psfsub/kb220131_66_tr_psfsub_vcubes.fits
